In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import scsavailability as scs
    
import statsmodels.api as sm
from scipy import stats

from scsavailability import features as feat, model as md, plotting as pt, score as sc

2020-12-17 13:38:05,053 - NumExpr defaulting to 8 threads.


In [2]:
path = '../raw_data/'

In [3]:
at = pd.read_csv(path+'active_totes_20201123.csv')
av = pd.read_csv(path+'Availability_with_Grey&Blue_1811-2511.csv')
#fa = pd.read_csv(path + 'scs_tote_matched.csv')
scs_raw = pd.read_csv('../raw_data/SCS alerts Nov.csv')

In [4]:
fa = feat.add_code(scs_raw)
fa, unmapped = feat.add_tote_colour(fa)

Running: Function "load_tote_lookup" (id=2475968027376) was called 1 times


In [5]:
at = feat.pre_process_AT(at)

Running: Function "pre_process_AT" (id=2475968026224) was called 1 times


In [6]:
av = feat.pre_process_av(av)

Running: Function "pre_process_av" (id=2475968027088) was called 1 times


In [7]:
fa = feat.preprocess_faults(fa,remove_same_location_faults = True)

Running: Function "preprocess_faults" (id=2475968028240) was called 1 times
Running: Function "load_module_lookup" (id=2475968025504) was called 1 times
duplicated location faults removed - max duration kept
HOTFIX: Quadrant only faults, PTT Asset Code update


In [40]:
fa_floor = feat.floor_shift_time_fa(fa, shift=0)

Time shifted by 0Minutes


In [71]:
mod = [1]
limit = 200

In [104]:
fa_sel_1 = feat.get_data_faults(fa_floor, modules = mod)

most_common = list(((fa_sel_1['Asset Code'].value_counts() > 250)[fa_sel_1['Asset Code'].value_counts() > 250]).index)
fa_sel = fa_sel_1[fa_sel_1['Asset Code'].isin(most_common)]

In [92]:
#fa_sel = feat.fault_select(fa_floor, fault_select_options = {'Tote Colour':['Blue']})
                                                            
fa_agg = feat.faults_aggregate(fa_sel, fault_agg_level= 'Asset Code', agg_type = 'count')

In [93]:
av_sel, at_sel = feat.av_at_select(av, at, remove_high_AT = True, availability_select_options = {"Module" : mod})

av_agg = feat.aggregate_availability(av_sel, agg_level = 'Module')
at_agg = feat.aggregate_totes(at_sel, agg_level = 'Module')

Running: Function "aggregate_availability" (id=2475968025936) was called 8 times


In [94]:
df = feat.merge_av_fa_at(av_agg ,at_df=at_agg, fa_df = fa_agg, agg_level = 'None')

In [95]:
X,y = md.gen_feat_var(df)
# Flip to availability
y=1-y

In [97]:
X_train, X_test, y_train, y_test = md.split(X,y)

#RF_mdl,predictions_RF=md.run_RF_model(X_train, X_test, y_train, y_test,num_trees=100, criterion = 'mse', max_depth=None, dtree=False)

Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test)

XGB_mdl,predictions_XGB,Importance_XGB = md.run_XGB_model(X_train, X_test, y_train, y_test)

maxf = None

# X_sel_rf = md.select_features(X, y, RF_mdl, thres = -np.inf, max_feat = maxf)

#X_sel_lm = md.select_features(X, y, Linear_mdl, selection_options={'thres': 'mean', 'max_feat': None})

#X_sel_XGB = md.select_features(X, y, XGB_mdl, selection_options={'thres': 'mean', 'max_feat': None})

#X_train, X_test, y_train, y_test = md.split(X_sel_lm,y,test_size=0.3,random_state=101)

#Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test)


cv_R2 = md.cross_validate_r2(Linear_mdl, X, y, n_folds = 5, shuffle = True, random_state = 101)

#cv_R2 = md.cross_validate_r2(RF_mdl, X_sel_rf, y, n_folds = 10, shuffle = True, random_state = 101)

cv_R2 = md.cross_validate_r2(XGB_mdl, X, y, n_folds = 10, shuffle = True, random_state = 101)

#md.stats_model(Linear_mdl,X_test,y_test)

Feature Importance Ranking: 
 
    Importance      Feature
0    0.214101        TOTES
1    0.148478  C2302STA014
2    0.129058  C2301RDC061
3    0.083377       SCS011
4    0.076334  C0503STA252
5    0.065500  C1603BCL054
6    0.058668  C2301BCL177
7    0.047371  C1603STA253
8    0.044931    C05PTT012
9    0.039630  C1603BCL014

Regression Metrics: 
 
          XGB Metrics
MAE         0.132136
MSE        0.0264231
RMSE        0.162552
MAPE%          28.52
ACC%           71.48
OOB              N/A
R2_Train    0.999965
R2_Pred     0.447562 


Cross Validation Scores LinearRegression(): 
 
        R2 Scores
1       0.429750
2       0.205244
3       0.395064
4       0.548279
5       0.378044
Mean    0.391276
STD     0.110388

Cross Validation Scores XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012,

In [98]:
import statsmodels.api as sm

In [99]:
model = sm.OLS(y,X)
results = model.fit()

In [100]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               Downtime   R-squared:                       0.563
Model:                            OLS   Adj. R-squared:                  0.513
Method:                 Least Squares   F-statistic:                     11.17
Date:                Thu, 17 Dec 2020   Prob (F-statistic):           5.11e-17
Time:                        14:00:08   Log-Likelihood:                 91.569
No. Observations:                 146   AIC:                            -151.1
Df Residuals:                     130   BIC:                            -103.4
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
C0502STA077     0.0357      0.085      0.423      

In [105]:
rm =  ['C0502STA077', 'C05PTT011', 'C1603BCL054', 'C2301BCL177', 'C2301RDC173', 'Module']

In [106]:
X_d = X.drop(rm, axis=1)

In [107]:
model = sm.OLS(y,X_d)
results = model.fit()
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:               Downtime   R-squared (uncentered):                   0.938
Model:                            OLS   Adj. R-squared (uncentered):              0.933
Method:                 Least Squares   F-statistic:                              204.1
Date:                Thu, 17 Dec 2020   Prob (F-statistic):                    1.02e-76
Time:                        14:49:46   Log-Likelihood:                          81.068
No. Observations:                 146   AIC:                                     -142.1
Df Residuals:                     136   BIC:                                     -112.3
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------

### Add LM - metrics

In [108]:
#sc.lm_coefficients(Linear_mdl, X, y)

In [109]:
X_train = sm.add_constant(X_train)
model = sm.OLS(y_train,X_train)
results = model.fit()
print(results.summary())

X_new = sm.add_constant(X_test)
#y_prediction =  results.predict(X_test)

# plt.scatter(y_prediction,y_test)

# from sklearn.metrics import r2_score
# print(r2_score(y_prediction,y_test))

                            OLS Regression Results                            
Dep. Variable:               Downtime   R-squared:                       0.566
Model:                            OLS   Adj. R-squared:                  0.490
Method:                 Least Squares   F-statistic:                     7.462
Date:                Fri, 18 Dec 2020   Prob (F-statistic):           2.85e-10
Time:                        08:01:00   Log-Likelihood:                 71.117
No. Observations:                 102   AIC:                            -110.2
Df Residuals:                      86   BIC:                            -68.24
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
C0502STA077     0.0343      0.083      0.414      

In [ ]:
# plt.figure(figsize=(20,5))
# sns.barplot(data=Coeff, x='Feature',y = 'Coefficients')

In [ ]:
plt.scatter(predictions_LM,y_test)